In [ ]:
# !pip install gensim
# !pip install nmslib
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install
# !pip install sentence_transformers
# !pip install annoy
# !pip install nltk
# !pip install contractions
import contractions
import nmslib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import time



sns.set()

In [ ]:
# data = pd.read_csv("netflix_titles.csv")
data = pd.read_excel("Data to be worked upon - Cohesity.xlsx")
data.dropna(subset=["Activity Detail"], inplace=True)
data.drop_duplicates(subset=["Activity Detail"], inplace=True)
data

,Session Identifier,Email,Searches,Clicks,Cases Logged,Support Visit,Session Start Time,Session last Activity Time,Activity Type,Activity Detail,...,Search Result Count,Search Result Clicked,Page No.,Facet Id,Facet Type,Facet Value,Exact Phrase,With one or more,Without the Words,Activity Time
0,1654046030084722,stephen.troy@envestnet.com,5,2,1,Visited Support,2022-06-01 06:43:50,2022-06-01 07:36:11,Text Searched,add new node to cluster,...,10,Yes,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-01 06:44:15
2,1654046030084722,stephen.troy@envestnet.com,5,2,1,Visited Support,2022-06-01 06:43:50,2022-06-01 07:36:11,Text Searched,add new cohesity node,...,7504,No,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-01 07:01:12
3,1654046030084722,stephen.troy@envestnet.com,5,2,1,Visited Support,2022-06-01 06:43:50,2022-06-01 07:36:11,Text Searched,add new cohesity node,...,7504,Yes,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-01 07:11:44
4,1654046030084722,stephen.troy@envestnet.com,5,2,1,Visited Support,2022-06-01 06:43:50,2022-06-01 07:36:11,Text Searched(Support Page),cohesity cluster was unable to connect to agent.,...,100,No,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-01 07:35:25
5,1654049631641179,alexander.nacino@teletech.com,1,1,1,Visited Support,2022-06-01 07:43:54,2022-06-01 07:59:23,Text Searched(Support Page),attachdisks failed: rpc call attachdiskstogues...,...,4614,Yes,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-01 07:45:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,1657896828090294,annette.marzolf@bcbsnd.com,2,1,1,Visited Support,2022-07-15 20:23:48,2022-07-15 20:53:26,Text Searched(Support Page),alert code ce03801007 disk free space is lo...,...,100,No,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-15 20:53:19
2982,1657906931051402,cohesity-storage@cdw.com,2,1,1,Visited Support,2022-07-15 23:12:11,2022-07-15 23:21:34,Text Searched(Support Page),expired certificate 192.168.8.74 cohesity inc ...,...,8316,Yes,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-15 23:17:56
2984,1657908423907099,ricky.stuckwish@expedient.com,3,2,1,Visited Support,2022-07-15 23:37:04,2022-07-16 00:40:53,Text Searched,backup task failed with error: [kagenterrornot...,...,10,Yes,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-15 23:42:05
2985,1657908423907099,ricky.stuckwish@expedient.com,3,2,1,Visited Support,2022-07-15 23:37:04,2022-07-16 00:40:53,Text Searched,failed to record the backup in backupset table...,...,10,Yes,1,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-15 23:42:23


In [ ]:
# def remove_num(query):
#   print(type(query))
#   if str(type(query)) != "<class 'int'>":
#     return True
#   return False

In [ ]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stops = stopwords.words('english')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings('ignore')

stopwords_eng = stopwords.words('english')


def remove_stops(query):
  try:
    new_word = []
    for word in query.split():
        if word in stopwords_eng or word.isdigit():
            pass
        else:
            new_word.append(word.strip())
    
    return ' '.join(new_word)
  
  except AttributeError as e:
    return False

def preprocessing(text):
  #to lowercase
    text = str(text).lower()
    #removing unnecessary dictionaries and braces
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\{[^)]*\}', '', text)
    #remove new lines
    text = text.replace('\\n',' ')
    #replacement
    text = text.replace(re.escape("\]\["), "")
    text = text.replace(re.escape("]"), "")
    #remove none meaningful    
    text = re.sub(r'\\x[0-9a-f]{2}', '',text)
    #remove emails
    text = re.sub(r'\S*@\S*\s?',' ',text)
    #remove mentions
    text = re.sub(r'@\S+', ' ', text)
    #contractions
    text = contractions.fix(text)
    #remove hashtags
    text = re.sub(r'@\S+', ' ', text)
    #remove emojis
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    #remove retweets
    text = text.replace(r'rt', '')
    #remove all punct
    text = re.sub('[^A-za-z0-9]', ' ', text)
    #remove extras whitespaces
    text = re.sub('\s+', ' ', text)

    text = text.replace("[","").replace("]","")
    return text

# data = data[data["Activity Detail"].apply(lambda x: not str(x).isdigit())]
data["clean_subject"] = data["Activity Detail"].apply(preprocessing)
data["clean_subject"] = data["clean_subject"].apply(remove_stops)

In [ ]:
data = data[data["clean_subject"] != False]

In [ ]:

# data = data.sample(25)

In [ ]:
# import pandas as pd 
# import numpy as np
# import re
# from annoy import AnnoyIndex
# from sentence_transformers import SentenceTransformer
# import contractions
# df1 = pd.read_excel('Datasets/Support Data.xlsx', sheet_name = 'Search_with_no_clicks')
# df2 = pd.read_excel('Datasets/Support Data.xlsx', sheet_name = 'Search_with_no_clicks_resulted_')
# df1 = df1[['Activity Detail','Email']]
# df2 = df2[['Activity Detail','Email']]
# print(df1.shape,df2.shape)
# concat_list  = [df1,df2]
# merged_df = pd.concat(concat_list)
# len(merged_df['Activity Detail'].unique())
# df2.isnull().sum()
# df1.head()
# df2.head()
# # Preprocessing ,empbedding ,loading embedding, recommender dataframe generation ,grouping
# def recommendor_algo(data):
#     print('Shape before Duplicate :',data.shape)
#     data = data['Activity Detail'].drop_duplicates().reset_index()
#     print('Shape after Deleting Duplicatres',data.shape)
    
#     model = SentenceTransformer('all-mpnet-base-v2')
#     encoding_ = model.encode(data['Activity Detail'])
#     data_encoding = np.array(encoding_)
#     np.save('encoding.npy', data_encoding)
#     embeding = np.load('encoding.npy')
#     print('='*70)
#     print(f'Loaded a DataFrame with {len(data)} rows and an embeddings matrix of dimensions {embeding.shape}')
    
#     search_index = AnnoyIndex(embeding.shape[1], 'angular')
#     for i in range(len(embeding)):
#         search_index.add_item(i, embeding[i])
#     search_index.build(10) 
    
#     df = data
#     data_list = []
#     for ids in range(1,embeding.shape[0]):
        
#         similar_item_ids = search_index.get_nns_by_item(ids,50,search_k = -1,include_distances=True)
        
#         data = pd.DataFrame(data={'Activity Detail':df.iloc[ids]['Activity Detail'],'Match_Act_Detail': df.iloc[similar_item_ids[0]]['Activity Detail'],
#                              'Distance': similar_item_ids[1]}).drop(ids)
#         # print(data)
#         data_list.append(data)
#     appended_data = pd.concat(data_list) 
#     appended_data['Distance'] = appended_data['Distance'].round(1)
    
#     data_threshold = appended_data[(appended_data['Distance'] <= 0.8 )]
#     data_outlier   = appended_data[(appended_data['Distance'] > 0.8 )]
    
#     #appended_data.drop("index", axis=1, inplace=True)
    
#     print('Threshold',data_threshold.shape,'Outlier',data_outlier.shape)
#     #Group by
#     data_threshold_group = data_threshold.groupby(['Activity Detail','Distance'])
#     data_outlier_group = data_outlier.groupby(['Activity Detail','Distance'])
    

#     return data_threshold_group , data_outlier_group
    
# def save_file(data,Filename):
#     data.first().to_excel(str(Filename) +'.xlsx',encoding='UTF-8')
# # Recommendor Algo
# threshold,outlier = recommendor_algo(df2)
# # Saving Grouped File 
# save_file(threshold,'Cases_No_clicks_Ram_domain_threshold')
# save_file(outlier,'Cases_No_clicks_Ram_domain_outlier')
# merged_df.shape


In [ ]:
import annoy
from sentence_transformers import SentenceTransformer

In [ ]:
class NMSLIBIndex():
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[0]
    self.vectors = vectors.astype('float32')
    self.labels = labels
  
  def build(self):
    self.index = nmslib.init(method='hnsw', space='cosinesimil')
    self.index.addDataPointBatch(self.vectors)
    self.index.createIndex({'post': 2})
        
  def query(self, vector, k=100):
    indices = self.index.knnQuery(vector, k=k)
    return [self.labels[i] for i in indices[0]], [round(i,2) for i in indices[1]]


class AnnoyIndex():
  def __init__(self, vectors, labels):
    self.dimension = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels    
    self.search_in_x_trees = 8

  def build(self, number_of_trees=5):
    self.index = annoy.AnnoyIndex(self.dimension)
    for i, vec in enumerate(self.vectors):
        self.index.add_item(i, vec.tolist())
    self.index.build(number_of_trees)
    
  def query(self, vector, k=100):
    indices = self.index.get_nns_by_vector(
          vector.tolist(), 
          k, 
          search_k=self.search_in_x_trees,
          include_distances= True)                                           
    return [self.labels[i] for i in indices[0]], [round(i,2) for i in indices[1]]

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')
encoding_ = model.encode(data['clean_subject'].to_list())
data_encoding = np.array(encoding_)
np.save('encoding.npy', data_encoding)
embedding = np.load('encoding.npy')
# print(embedding)

In [ ]:
# for i in range(len(embedding)):
        # index.addDataPoint(i, embedding[i])

In [ ]:
vector_list = []
for emb in embedding:
  vector_list.append(emb)

vect = np.array(vector_list)
# vect = np.stack(vector_list, axis=0 )

# vect = np.reshape(vect, (-1,1))
len(vector_list)
# vect.shape

2305

In [ ]:
new_df = pd.DataFrame({"vector" : vector_list, "label" : data["clean_subject"]})
new_df = new_df.reset_index()
new_df.drop(columns=["index"],inplace=True)
new_df

,vector,label
0,"[-0.029099623, -0.0685533, -0.015035429, -0.02...",add new node cluster
1,"[-0.039144177, -0.047255028, 0.009903121, -0.0...",add new cohesity node
2,"[-0.039144177, -0.047255028, 0.009903121, -0.0...",add new cohesity node
3,"[-0.017813697, -0.07626766, 0.017513614, -0.05...",cohesity cluster unable connect agent
4,"[0.026774924, 0.024233818, 0.04761586, -0.0445...",attachdisks failed rpc call attachdiskstoguest...
...,...,...
2300,"[0.02241886, -0.026540743, -0.004749271, -0.01...",ale code ce03801007 disk free space low node i...
2301,"[-0.01498969, -0.009075889, 0.02440182, -0.035...",expired ceificate cohesity inc server ssl secu...
2302,"[0.0071966047, -0.021235924, -0.018005181, -0....",backup task failed error kagenterrornotretryab...
2303,"[0.014289275, -0.0009379565, 0.010814664, 0.00...",failed record backup backupset table mssqlserv...


In [ ]:
start = time.time()
# --------------------------------

nm_index = NMSLIBIndex(vect, new_df["label"])
nm_index.build()

# --------------------------------
print("Process time: " + str(round(time.time() - start,3)) +"s")

Process time: 2.368s


In [ ]:
start = time.time()
# --------------------------------

an_index = AnnoyIndex(vect, new_df["label"])
an_index.build()

# --------------------------------
print("Process time: " + str(round(time.time() - start,3)) +"s")

Process time: 0.117s


In [ ]:
start = time.time()
# --------------------------------

queries_nm, distances_nm = nm_index.query(new_df['vector'][100])

nm_df = pd.DataFrame({
    "query_nm" : queries_nm,
    "distance_nm" : distances_nm
})


# --------------------------------
print("Process time: " + str(round(time.time() - start,3)) +"s")

Process time: 0.003s


In [ ]:
start = time.time()
# --------------------------------

queries_an, distances_an = an_index.query(new_df['vector'][100])

an_df = pd.DataFrame({
    "query_an" : queries_an,
    "distance_an" : distances_an
})

# --------------------------------
print("Process time: " + str(round(time.time() - start,3)) +"s")


Process time: 0.008s


In [ ]:
print(f"Main Query - ", new_df['label'][100])

Main Query -  vm checkpoint creation failed


In [ ]:
pd.concat([nm_df,an_df], axis=1).head(30)

,query_nm,distance_nm,query_an,distance_an
0,vm checkpoint creation failed,0.00,vm checkpoint creation failed,0.00
1,vm checkpoint creation failed khypervcmdexcept...,0.14,vm checkpoint creation failed khypervcmdexcept...,0.53
2,vmware backup error template vm,0.44,vmware backup error template vm,0.94
3,vm backup fails,0.44,vm backup fails,0.94
4,vm restore failed code one parameters invalid ...,0.45,vm restore failed code one parameters invalid ...,0.95
5,io filter coh_bootbank_cohfilter_1 1oem config...,0.46,io filter coh_bootbank_cohfilter_1 1oem config...,0.96
6,vm failing back crash consistent snapshot sinc...,0.46,vm failing back crash consistent snapshot sinc...,0.96
7,backup task failed error kvixerror code unknow...,0.46,backup task failed error kvixerror code unknow...,0.96
8,creating restore file failed error http error ...,0.47,creating restore file failed error http error ...,0.97
9,unable restore vm code,0.48,unable restore vm code,0.98


In [ ]:
# making clusters

In [ ]:
queries_nm, distances_nm = nm_index.query(new_df['vector'][100])